<h1><b>Markov Decision Processes</h1></b>
<p align="justify">Στη συγκεκριμένη άσκηση θα μελετήσετε τους αλγορίθμους <i>Policy Iteration</i> και <i>Value Iteration</i>, καθώς και θα εξοικειωθείτε με βασικές έννοιες των <i>Markov Decision Processes</i>. Οι αλγόριθμοι <i>Policy Iteration</i> και <i>Value Iteration</i> είναι από τους βασικούς αλγορίθμους δυναμικού προγραμματισμού που χρησιμοποιούνται για την επίλυση της εξίσωσης <i>Bellman</i> σε <i>Markov Decision Processes</i>.</p>
<p align="justify">Το πρόβλημα που θα μελετήσετε είναι αυτό της παγωμένης λίμνης (Frozen Lake) με μέγεθος πλέγματος 8 x 8.</p>


<h2><b>Εξοικείωση με τη βιβλιοθήκη <i>Gym</i></b></h2>

In [ ]:
!pip install gymnasium
import numpy as np
import gymnasium as gym
from gym import wrappers

Με την παρακάτω εντολή, ορίζετε το πρόβλημα που θα μελετηθεί:

In [9]:
env_name = 'FrozenLake-v1'
env = gym.make(env_name, map_name='8x8', render_mode='rgb_array')

Με τις παρακάτω εντολές, θα επαναφέρετε τον Agent στην αρχική του θέση και θα οπτικοποιήσετε το πλέγμα και τη θέση του Agent

In [ ]:
env.reset()
env.render()

Με τις παρακάτω εντολές, ορίζετε την επόμενη ενέργεια με τυχαίο τρόπο και ο Agent κάνει ένα βήμα.

In [ ]:
next_action = env.action_space.sample()
env.step(next_action)
env.render()

Να εκτελέσετε αρκετές φορές τις τελευταίες εντολές και να παρατηρήσετε κάθε φορά την ενέργεια που ζητείται από τον agent να εκτελέσει και την ενέργεια που αυτός πραγματοποιεί. Πραγματοποιεί πάντοτε ο agent την κίνηση που του ζητείται; Είναι ντετερμινιστικές ή στοχαστικές οι κινήσεις του agent;

<h2><b>Ερωτήσεις</b></h2>

<ul>
<li>Μελετώντας <a href="https://machinelearningjourney.com/index.php/2020/07/02/frozenlake/">αυτό</a> και βασισμένοι στα συμπεράσματα του προηγούμενου ερωτήματος να περιγράψετε σύντομα το πρόβλημα της παγωμένης λίμνης ως πρόβλημα βελτιστοποίησης. Ποιος είναι ο στόχος του agent;</li>
<li>Να διατυπώσετε την ιδιότητα <i>Markov</i>. Πώς απλοποιεί η ιδιότητα αυτή τη μελέτη του συγκεκριμένου προβλήματος;</li>
<li>Να περιγράψετε σύντομα τους αλγορίθμους <i>Policy Iteration</i> και <i>Value Iteration</i>, δίνοντας έμφαση στο διαφορετικό τρόπο με τον οποίο προσεγγίζουν την επίλυση του προβλήματος. Είναι εγγυημένο ότι οι δύο αλγόριθμοι θα συγκλίνουν στη βέλτιστη πολιτική; Αν ναι, οδηγούν σε ίδια ή διαφορετική βέλτιστη πολιτική;</li>
<li>Να εκτελέσετε τα προγράμματα που σας δίνονται, τα οποία επιλύουν το
πρόβλημα της παγωμένης λίμνης, χρησιμοποιώντας τους αλγορίθμους <i>Policy
Iteration</i> και <i>Value Iteration</i> αντίστοιχα. Ποια μέθοδος συγκλίνει στη βέλτιστη λύση σε λιγότερα βήματα; Τι συμπέρασμα βγάζετε; Να ελέγξετε το χρόνο εκτέλεσης του κάθε προγράμματος, χρησιμοποιώντας την εντολή <i>time</i>. Τι συμπέρασμα βγάζετε ως προς την πολυπλοκότητα του κάθε αλγορίθμου;</li>
</ul>

<h2><b>Policy Iteration</b></h2>

In [ ]:
"""
Solving FrozenLake8x8 environment using Policy iteration.
Based on the implementation of Moustafa Alzantot (malzantot@ucla.edu)
"""
import numpy as np
import gym
from gym import wrappers


def run_episode(env, policy, gamma = 1.0, render = False):
    """ Runs an episode and return the total reward """
    obs = env.reset()
    total_reward = 0
    step_idx = 0
    while True:
        if render:
            env.render()
        obs, reward, done , _ , _= env.step(int(policy[obs]))
        total_reward += (gamma ** step_idx * reward)
        step_idx += 1
        if done:
            break
    return total_reward


def evaluate_policy(env, policy, gamma = 1.0, n = 100):
    scores = [run_episode(env, policy, gamma, False) for _ in range(n)]
    return np.mean(scores)

def extract_policy(v, gamma = 1.0):
    """ Extract the policy given a value-function """
    policy = np.zeros(env.observation_space.n) # Modified to use observation_space.n
    for s in range(env.observation_space.n):   # Also modified here
        q_sa = np.zeros(env.action_space.n)    # Using action_space.n
        for a in range(env.action_space.n):
            q_sa[a] = sum([p * (r + gamma * v[s_]) for p, s_, r, _ in  env.P[s][a]])
        policy[s] = np.argmax(q_sa)
    return policy

def compute_policy_v(env, policy, gamma=1.0):
    """ Iteratively evaluate the value-function under policy. """
    v = np.zeros(env.observation_space.n) # Modified to use observation_space.n
    eps = 1e-10
    while True:
        prev_v = np.copy(v)
        for s in range(env.observation_space.n): # Also modified here
            policy_a = policy[s]
            v[s] = sum([p * (r + gamma * prev_v[s_]) for p, s_, r, _ in env.P[s][policy_a]])
        if (np.sum((np.fabs(prev_v - v))) <= eps):
            # value converged
            break
    return v

def policy_iteration(env, gamma = 1.0):
    """ Policy-Iteration algorithm """
    # Using env.action_space.n for the number of actions
    policy = np.random.choice(env.action_space.n, size=(env.observation_space.n))  # initialize a random policy
    max_iterations = 200000
    gamma = 1.0
    for i in range(max_iterations):
        old_policy_v = compute_policy_v(env, policy, gamma)
        new_policy = extract_policy(old_policy_v, gamma)
        if (np.all(policy == new_policy)):
            print ('Policy-Iteration converged at step %d.' %(i+1))
            break
        policy = new_policy
    return policy

if __name__ == '__main__':
    env_name  = 'FrozenLake8x8-v1'
    env = gym.make(env_name)
    env = env.unwrapped
    optimal_policy = policy_iteration(env, gamma = 1.0)
    scores = evaluate_policy(env, optimal_policy, gamma = 1.0)
    print('Average scores = ', scores)  # Removed np.mean since evaluate_policy already returns the mean score

<h2><b>Value Iteration</b></h2>

In [ ]:
"""
Solving FrozenLake8x8 environment using Value-Itertion.
Based on the implementation of Moustafa Alzantot (malzantot@ucla.edu)
"""
import numpy as np
import gym
from gym import wrappers


def run_episode(env, policy, gamma=1.0, render=False):
    """Runs an episode and returns the total reward."""
    obs = env.reset()
    total_reward = 0
    step_idx = 0
    while True:
        if render:
            env.render()
        obs, reward, done, _ , _ = env.step(int(policy[obs]))
        total_reward += (gamma ** step_idx * reward)
        step_idx += 1
        if done:
            break
    return total_reward

def evaluate_policy(env, policy, gamma=1.0, n=100):
    """Evaluates a policy by running it n times."""
    scores = [run_episode(env, policy, gamma, False) for _ in range(n)]
    return np.mean(scores)

def extract_policy(v, gamma, env):
    """Extracts the policy given a value-function."""
    policy = np.zeros(env.observation_space.n, dtype=int)
    for s in range(env.observation_space.n):
        q_sa = np.zeros(env.action_space.n)
        for a in range(env.action_space.n):
            for p, s_, r, _ in env.P[s][a]:
                q_sa[a] += (p * (r + gamma * v[s_]))
        policy[s] = np.argmax(q_sa)
    return policy

def value_iteration(env, gamma=1.0):
    """Performs value iteration."""
    v = np.zeros(env.observation_space.n)  # initialize value-function
    max_iterations = 100000
    eps = 1e-20
    for i in range(max_iterations):
        prev_v = np.copy(v)
        for s in range(env.observation_space.n):
            q_sa = [sum([p * (r + gamma * prev_v[s_]) for p, s_, r, _ in env.P[s][a]]) for a in range(env.action_space.n)]
            v[s] = max(q_sa)
        if (np.sum(np.fabs(prev_v - v)) <= eps):
            print(f'Value-iteration converged at iteration# {i+1}.')
            break
    return v

if __name__ == '__main__':
    env_name = 'FrozenLake8x8-v1'  # Make sure to use the correct environment version
    gamma = 1.0
    env = gym.make(env_name)
    env = env.unwrapped  # Note: Unwrapping might not always be necessary or desired
    optimal_v = value_iteration(env, gamma)
    policy = extract_policy(optimal_v, gamma, env)
    policy_score = evaluate_policy(env, policy, gamma, n=1000)
    print('Policy average score = ', policy_score)
